# Data base queury to identify transcription start sites.

© 2022 Tom Röschinger. This work is licensed under a <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution License CC-BY 4.0</a>. All code contained herein is licensed under an <a href="https://opensource.org/licenses/MIT">MIT license</a>

***

In [60]:
using wgregseq, FASTX, DataFrames, CSV, BioSequences, Statistics, StatsBase, Dates

First we import the wildtype sequence for *E. coli* K12 MG1655. 

In [61]:
# Import genome
re = open(FASTA.Reader, "../data/ecocyc/mg1655_genome.fasta")
wt_sequence = [sequence(record) for record in re][1]

4641652nt DNA Sequence:
AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGT…AATATCACCAAATAAAAAACGCCTTAGTAAGTATTTTTC

Next we import the list of genes that we want to create sequences for. Once the genes are imported, we group them into certain classes.

In [62]:
# Import gene list to generate sequences for
gene_table = CSV.read(
    "../data/100_genes.csv", 
    DataFrames.DataFrame, 
    delim=",",
    comment="#",
    missingstring="none",
)

# Give IDs to groups for adding primers later
group_dict = Dict{String, Int64}(filter(x -> occursin("Antibiotic/toxin", x), gene_table.group) .=> 1)
group_dict["Gold Standard"] = 2
group_dict["Heinemann dataset"] = 3
group_dict["uncharacterized protein"] = 4
group_dict["Heinemann dataset uncharacterized"] = 7
group_dict["YmfT_modulon"] = 5
group_dict["YgeV_modulon"] = 6


groups = zeros(nrow(gene_table))
for i in 1:nrow(gene_table)
    if gene_table[i, "group"] in keys(group_dict)
        groups[i] = group_dict[gene_table[i, "group"]]
    else
        groups[i] = 8
    end
end

# Add IDs to table
insertcols!(gene_table, 4, :group_ID => groups)

# Print number of genes per group
println(combine(groupby(gene_table, "group"), nrow))

14×2 DataFrame
 Row │ group                              nrow  
     │ String                             Int64 
─────┼──────────────────────────────────────────
   1 │ Gold Standard                         18
   2 │ Antibiotic/toxin                      19
   3 │ Inc-4 (x) - stress                     1
   4 │ Inc-4 (y) - stress                     1
   5 │ Inc-4 (z) - stress                     1
   6 │ Inc-1 (x)                              1
   7 │ Inc-1 (y)                              1
   8 │ Inc-1 (z1)                             1
   9 │ Inc-1 (z2)                             1
  10 │ Heinemann dataset                      9
  11 │ Heinemann dataset uncharacterized     11
  12 │ uncharacterized protein               13
  13 │ YmfT_modulon                          16
  14 │ YgeV_modulon                          16


Import the promoters that we collected from the databases.

In [63]:
# Import promoter list and infer types that can be infered automatically
promoter_list = CSV.read(
    #"../data/promoter_list_processed.csv", 
    "../data/promoter_list_ecocyc.csv",
    DataFrames.DataFrame, 
    types=Dict(
        "promoter"=>String,
        "tss"=>Float64,
        "direction"=>String
    )
)
first(promoter_list, 5)

,promoter,genes,gene_position,direction
,String,String,String,String?
1,uspAp1,"[""uspA""]",[3.640111e6],+
2,glnBp3,"[""glnB""]",[2.687408e6],-
3,mazEp2,"[""mazG"", ""mazE"", ""mazF""]","[2.910685e6, 2.911339e6, 2.911091e6]",-
4,ffhp,"[""ffh""]",[2.747795e6],-
5,blrp,"[""blr""]",[1.704551e6],+


Import the table of genes that we could not identify a promoter for.

In [64]:
operons_without_promoters = CSV.read(
    "../data/operons_without_promoters.csv", 
    DataFrames.DataFrame, 
    types=Dict(
        "direction"=>String
    )
)

first(operons_without_promoters, 5)

,genes,direction,gene_position
,String,String,String
1,"[""tfaE"", ""stfE""]",-,"[1.209119e6, 1.209655e6]"
2,"[""yabR""]",-,[85511.0]
3,"[""trmB"", ""yggL""]",-,"[3.102852e6, 3.102133e6]"
4,"[""rluB""]",+,[1.326852e6]
5,"[""insH2""]",-,[575717.0]


Importing dataframes with columns that contain arrays as entries leads to them being imported as strings.

In [65]:
typeof(promoter_list.gene_position)  # Vector{String}

Vector{String} (alias for Array{String, 1})

To be able to work with these columns we need to transform the types back to their correct type. We wrote some extensions to the `parse` function in Julias `Base`, which does exactly that. Make sure that the `wgregseq` package is imported, such that these functions are available for use.

In [66]:
# Replace columns by nicer types
promoter_list.genes = parse.(Vector{String}, promoter_list.genes)
promoter_list.gene_position = parse.(Vector{Float64}, promoter_list.gene_position)
promoter_list.evidence = parse.(Vector{String}, promoter_list.evidence)

operons_without_promoters.genes = parse.(Vector{String}, operons_without_promoters.genes)
operons_without_promoters.gene_position = parse.(Vector{Float64}, operons_without_promoters.gene_position)

typeof(promoter_list.gene_position)  # Vector{Vector{Float64}}

Vector{Vector{Float64}} (alias for Array{Array{Float64, 1}, 1})

Import list of all genes to check for possible synomyms. Fix types of columns as well.

In [30]:
## Some genes may have the wrong synomym
all_gene_list = CSV.read(
    "../data/all_genes_table.csv", 
    DataFrame, 
    types=Dict(
        "ID"=>String,
        "gene"=>String,
        "gene_position"=>Float64,
        "direction"=>String
    )
)

# Get the correct types
all_gene_list.synonyms = parse.(Vector{String}, all_gene_list.synonyms)
all_gene_list.transcription_units = parse.(Vector{String}, all_gene_list.transcription_units)

4675-element Vector{Vector{String}}:
 ["TU0-13152"]
 []
 ["TU0-14439", "TU0-941"]
 ["TU0-6982"]
 ["TU0-44222"]
 ["TU0-13809"]
 ["TU0-14803", "TU0-14802", "TU0-14535", "TU0-12859"]
 ["TU0-13209", "TU0-4404"]
 ["TU0-13195"]
 ["TU0-14363"]
 []
 ["TU0-8659"]
 ["TU0-13150"]
 ⋮
 ["TU0-7681"]
 ["TU0-8587"]
 ["TU0-8740"]
 ["TU0-13603"]
 ["TU0-8661"]
 ["TU0-13225"]
 ["TU0-8744"]
 ["TU0-13678"]
 ["TU0-13738"]
 ["TU0-13699"]
 ["TU0-14404", "TU0-13642"]
 ["none"]

Replace gene names with synomyms if necessary.

In [31]:
# Replace gene names if they are synonyms
for i in 1:nrow(gene_table)
    name = String(gene_table[i, "name"])
    if name ∉ all_gene_list.gene
        syn = all_gene_list[map(x -> name in x, all_gene_list.synonyms), :gene][1]
        gene_table[i, "name"] = syn
    end
end
gene_group_ID_dict = Dict(gene_table.name .=> gene_table.group_ID)

Dict{String7, Float64} with 106 entries:
  "yjbJ" => 7.0
  "ykgR" => 4.0
  "ykfM" => 4.0
  "ompR" => 8.0
  "ymfN" => 5.0
  "galE" => 8.0
  "sulA" => 5.0
  "tnaA" => 3.0
  "ysdE" => 1.0
  "yadE" => 4.0
  "crp"  => 8.0
  "yadM" => 4.0
  "tisB" => 1.0
  "ymfH" => 5.0
  "jayE" => 5.0
  "xdhA" => 6.0
  "emrB" => 1.0
  "lpp"  => 3.0
  "aceA" => 3.0
  "ssnA" => 6.0
  "ghoT" => 1.0
  "znuA" => 2.0
  "ihfA" => 8.0
  "dicA" => 2.0
  "emrA" => 1.0
  ⋮      => ⋮

Go through the genes for the experiment and find transcription start sites if there are some.

In [34]:
df_list = DataFrame[]
genes_no_tss = []
for gene in gene_table.name
    _df = promoter_list[map(x -> gene in x, promoter_list.genes), [:tss, :direction, :gene_position, :genes, :promoter, :evidence]]
    if nrow(_df) > 0
        push!(df_list, _df)
    else
        push!(genes_no_tss, gene)
    end
end
df = vcat(df_list...) |> unique
first(df, 5)

,tss,direction,gene_position,genes
,Float64,String?,Array…,Array…
1,1.65517e6,-,"[1.65514e6, 1.65392e6]","[""rspA"", ""rspB""]"
2,70075.0,-,"[66550.0, 70048.0, 68337.0]","[""araD"", ""araB"", ""araA""]"
3,1.94263e6,+,"[1.94266e6, 1.94341e6]","[""znuC"", ""znuB""]"
4,3.73081e6,-,"[3.72937e6, 3.73076e6]","[""xylB"", ""xylA""]"
5,3.73107e6,+,"[3.73498e6, 3.73372e6, 3.73113e6, 3.7322e6]","[""xylR"", ""xylH"", ""xylF"", ""xylG""]"


Find operons for genes that do not have a tss associated with them.

In [36]:
df_no_prom = DataFrame()
for gene in genes_no_tss
    append!(df_no_prom, operons_without_promoters[map(x -> gene in x, operons_without_promoters.genes), :])
end
if nrow(df_no_prom) > 0
    unique!(df_no_prom)
end

first(df_no_prom, 5)

,genes,direction,gene_position
,Array…,String,Array…
1,"[""dicA""]",+,[1.64793e6]
2,"[""yagB"", ""insX"", ""yagA""]",-,"[280735.0, 280362.0, 281983.0]"
3,"[""yjjJ""]",+,[4.62177e6]
4,"[""tmaR""]",-,[2.07936e6]
5,"[""yjbJ""]",+,[4.25924e6]


Import data set from Urtecho 2020 to find possible tss for genes.

In [38]:
# import Urtecho data
urtecho_tss = CSV.read(
    "../data/urtecho_2020/tss_operon_regulation.txt", 
    DataFrame 
)

first(urtecho_tss, 5)

,tss_name,tss_strand,tss_position,prom_expression,active
,String,String1,Int64,Float64,String15
1,TSS_11125_storz_regulondb,+,2945404,53.2139,active
2,TSS_16748_storz,+,4275469,0.748653,inactive
3,TSS_16918_storz,-,4339955,0.653776,inactive
4,TSS_9847_regulondb,+,2619175,0.801252,inactive
5,TSS_14431_storz,+,3718711,0.703256,inactive


Check if tss was identified for genes.

In [67]:
# Check if gene is part of operon
function occursin_operon(gene, operon)
    split_operon = split(operon, "-")
    a, b = gene[1:3], gene[4]
    return prod(occursin.(a, split_operon) .* occursin.(b, split_operon))
end

# Find operon in Urtecho data
df_tss_urtecho = DataFrame()

# Make array to store indeces of genes to be removed from list
delete_index_list = Int64[]
for i in 1:nrow(df_no_prom)
    # Get row data
    genes = df_no_prom[i, "genes"]
    gene_position = df_no_prom[i, "gene_position"]
    for gene in genes
        # Check for each gene if part of an identied operon
        operons = filter(x -> prod(occursin_operon.(gene, x)), urtecho_tss.operon)
        
        # Sanity check
        if (operons |> unique |> length) > 1
            throw(ErrorException("More than one operon for genes: $(genes)"))
            
        # Of no operon identified, skip
        elseif  (operons |> unique |> length) == 0

        # If one operon is identified, check for active site
        else
            operon = unique(operons)[1]
            temp = urtecho_tss[(urtecho_tss.operon .== operon) .& ((urtecho_tss.active .== "active")), :]
            # If active site is found, add to dataframeb
            if nrow(temp) != 0
                insertcols!(temp, 2, :genes => fill(genes, nrow(temp)))
                insertcols!(temp, 2, :gene_position => fill(gene_position, nrow(temp)))
                insertcols!(temp, 2, :evidence => fill(["EXP"], nrow(temp)))
                rename!(temp, "tss_strand" => "direction", "tss_position"=> "tss", "tss_name"=>"promoter")
                append!(df_tss_urtecho, temp)
                push!(delete_index_list, i)
            end
        end
    end
end

# Add found promoters to list
if nrow(df_tss_urtecho) > 0
    append!(df, df_tss_urtecho[:, ["genes", "tss", "direction", "gene_position", "promoter", "evidence"]])
end

# Remove genes with identified promoters from list of genes without promoters
df_no_prom = df_no_prom[Not(delete_index_list), :];

Find best predicted tss for genes that do not have a tss annotated using the Promoter Calculator from La Fleur et al 2022.

In [41]:
p = wgregseq.promoter_finder.Promoter_Calculator()
tss_list = Int64[]
name_list = String[]

for i in 1:nrow(df_no_prom)
    if df_no_prom[i, "direction"] == "+"
        ind = Int64(minimum(df_no_prom[i, "gene_position"]))
        sequence = wt_sequence[ind-500:ind]
        r = p(sequence)["Forward_Predictions_per_TSS"]
        _x = [(key, r[key]["dG_total"]) for key in keys(r) |> collect]
        tss = _x[argmin([x[2] for x in _x])][1] + ind - 500
    else
        ind = Int64(maximum(df_no_prom[i, "gene_position"]))
        sequence = wt_sequence[ind:ind+500]
        r = p(sequence)["Reverse_Predictions_per_TSS"] 
        _x = [(key, r[key]["dG_total"]) for key in keys(r) |> collect]
        tss = _x[argmin([x[2] for x in _x])][1] + ind
    end
    push!(tss_list, tss)

    push!(name_list, join(df_no_prom[i, "genes"], "_") * "_predicted")
end

if nrow(df_no_prom)  >0 
    # Add start sites to genes 
    insertcols!(df_no_prom, 2, :tss =>tss_list)
    # Add information that they are predicted
    insertcols!(df_no_prom, 5, :promoter =>name_list)
    insertcols!(df_no_prom, 5, :evidence =>fill(["COMP"], nrow(df_no_prom)))
    # Add promoters to list
    append!(df, df_no_prom)
end

,tss,direction,gene_position,genes
,Float64,String?,Array…,Array…
1,1.65517e6,-,"[1.65514e6, 1.65392e6]","[""rspA"", ""rspB""]"
2,70075.0,-,"[66550.0, 70048.0, 68337.0]","[""araD"", ""araB"", ""araA""]"
3,1.94263e6,+,"[1.94266e6, 1.94341e6]","[""znuC"", ""znuB""]"
4,3.73081e6,-,"[3.72937e6, 3.73076e6]","[""xylB"", ""xylA""]"
5,3.73107e6,+,"[3.73498e6, 3.73372e6, 3.73113e6, 3.7322e6]","[""xylR"", ""xylH"", ""xylF"", ""xylG""]"
6,1.64788e6,-,"[1.64764e6, 1.64732e6, 1.64785e6]","[""ydfX"", ""ydfW"", ""dicC""]"
7,1.6459e6,-,"[1.64563e6, 1.64587e6, 1.64527e6]","[""relE"", ""relB"", ""hokD""]"
8,932929.0,+,[933224.0],"[""ftsK""]"
9,933138.0,+,[933224.0],"[""ftsK""]"


Create mutated sequences for each transcription start site.

In [43]:
df_sequences = DataFrame()
for row in eachrow(df)
    tss = Int64(row.tss)
    direction = row.direction
    genes = row.genes
    promoter = row.promoter
    seq = wgregseq.design.find_seq(tss, direction, 115, 45, wt_sequence)[1]
    mut_list = wgregseq.design.mutations_rand(seq, 0.1, 1500)
    names = ["$(promoter)_$i" for i in 0:1500]
    _df = DataFrame(sequence=mut_list, genes=fill(genes, 1501), promoter=fill(promoter, 1501), name=names)
    global df_sequences = vcat(df_sequences, _df)
end

if any(length.(df_sequences.sequence) .!= 160)
    throw(ErrorException("Not all sequences are 160bp!"))
else
    println("Done!")
    println()
end
first(df_sequences, 5)

Done!



,sequence
,LongDNA
1,TTTCATCTTTTGTCAACCATTCACAGCGCAAATATACGCCTTTTTTTGTGATCACTCCGGCTTTTTTCGATCTTTATACTTGTATGGTAGTAGCTCAGTTGCGTAGATTTCATGCATCACGACAAGCGATGCAAGGAATCGAACATGAAGATCGTAAAGG
2,CTTCATCTTTTGTCAACCATTCACAACGCAAATATACGCTTTTTTTTGTGATCACTCCGGCTTTTTTCGATAATTATACTGGTCCGGTAGTAGCTCAGTTGCGTAGATTGAATGCATCACGACAAGTCATTAAAGTAACCGAACATGAAGATCGTAAAGG
3,TTTCATGTTTTGTCAAATATTCACAGCGCAAATATACGCCTTTTTTTGTGAACACTCCGGCTTATTTCGTTCTTTAGACTTGTATGGTGGTAGCTAAGTTCCGTAGACTTCATGCCTCACGACAAGCGATGCACGGGCTCGAACATAAAGATCGTAAAGG
4,TTTCATCTTTTGCCAGCCATTCACAGCGCAAATACTCCCCTTTTTCTATGATCACTCCGGCTTTGTTCGCTCTTTATACTTGTATGGTAGTAGCTCAGTTACGTAGATTTCATGTATCACGACAAGCTATGTAAGGAATCGAACATGATGATCTTAAGGG
5,TTTCATCTTTTGTCAACCATACACATCTCAAATAAAAACCTTTTTTTGTGATCACTCCCGCTTTTGTCAATCTTTATACTTTTATGGTAGTAGCTCAGCTGGGTAGATATCATGCATCACGACAAGCGATACAAGTAATCGAACATGAAGATCGTTAAGG


Do some exploration here of mutated sequences to show that there are what we think they are.

Add restriction sites.

In [ ]:
gdf = groupby(deepcopy(df_sequences), :genes)
df_stack = DataFrame()


enzymes = ["SalI", "SacI", "NheI", "XbaI", "SpeI", "XhoI", "EcoRI", "ApaI", "ScaI", "NcoI", "MluI", "EcoRV", "BbsI", "BamHI", "AgeI", "PstI", "NsiI", "SbfI"]
println("Adding restriction enzymes...")
for enz in enzymes
    if enz ∉ wgregseq.enzyme_list.enzyme
        throw(ErrorException("$enz is not in list of enzymes."))
    end
end


for group in gdf
    group[:, "sequence"] = wgregseq.design.add_primer(convert(Vector{LongSequence{DNAAlphabet{4}}}, (group.sequence)), 100, "both")
    df_restriction = wgregseq.design.find_restriction_sites(enzymes, group[:, "sequence"])
    sort!(df_restriction, "sites")
    dict = Dict{Any, Any}(df_restriction.enzyme .=> df_restriction.sites)
    dict["gene"] = [unique(group.genes)[1]]
    dict["promoter"] = [unique(group.promoter)[1]]
    append!(df_stack, DataFrame(dict))
end
df_stack

dict_enz = Dict{Any, Any}(enzymes .=> sum.(eachcol(df_stack[!, enzymes])))
dict_enz["gene"] = [String31["all"]]
dict_enz["promoter"] = "all"
append!(df_stack, DataFrame(dict_enz))

# Computational Environment

In [68]:
using Pkg
Pkg.status(["DataFrames", "CSV", "FASTX", "BioSequences"])

     Project wgregseq v0.1.0
      Status `~/git/1000_genes_ecoli/Project.toml`
  [7e6ae17a] BioSequences v3.0.0
  [336ed68f] CSV v0.10.2
  [a93c6f00] DataFrames v1.3.2
  [c2308a5c] FASTX v1.3.0
